In [ ]:
import duckrun
from   psutil import *

In [2]:
# please don't use a workspace name, Lakehouse and semantic_model with an empty space, or the same name of the lakehouse recently deleted
nbr_days_download     =  int(30 * 2 ** ((cpu_count() - 2) / 2))  # or just input your numbers
lh                    = 'power' 
schema                = 'aemo'
semantic_model        = "directlake_on_onelake" 
ws                    =  "temp"
Nbr_threads           = (cpu_count()*2)+1

In [ ]:
sql_folder            = 'https://github.com/djouallah/fabric_demo/raw/refs/heads/main/transformation/'

In [4]:
%%time
con = duckrun.connect(f"{ws}/{lh}.lakehouse/{schema}", sql_folder)

Connecting to Lakehouse...
Please login to Azure CLI
🔍 Discovering tables in schema 'aemo'...

📊 Found 9 Delta tables. Attaching as views...

🚀 Attempting batch view creation...
  ✅ Batch creation successful: 9 views created

✅ Successfully attached 9/9 tables

CPU times: total: 3.73 s
Wall time: 1min 4s


In [ ]:
intraday = [
              ('scrapingv2', (["http://nemweb.com.au/Reports/Current/DispatchIS_Reports/","http://nemweb.com.au/Reports/Current/Dispatch_SCADA/" ],
                            ["Reports/Current/DispatchIS_Reports/","Reports/Current/Dispatch_SCADA/"],
                             288, ws,lh,Nbr_threads)),
              ('price_today','append'),
              ('scada_today','append'),
              ('duid','ignore'),
              ('summary__incremental', 'append')            
          ]


In [6]:
%%time
#load today data
con.run(intraday)


Task 1/5: scrapingv2
Running Python: scrapingv2(['http://nemweb.com.au/Reports/Current/DispatchIS_Reports/', 'http://nemweb.com.au/Reports/Current/Dispatch_SCADA/'], ['Reports/Current/DispatchIS_Reports/', 'Reports/Current/Dispatch_SCADA/'], 288, 'temp', 'power', 17)
Flushed 1 files to disk and updated log Reports/Current/DispatchIS_Reports/download_log.csv
Flushed 2 files to disk and updated log Reports/Current/Dispatch_SCADA/download_log.csv
http://nemweb.com.au/Reports/Current/DispatchIS_Reports/ - 1 files extracted and uploaded
http://nemweb.com.au/Reports/Current/Dispatch_SCADA/ - 2 files extracted and uploaded
✅ Python 'scrapingv2' completed

Task 2/5: price_today
✅ SQL 'price_today' → 'price_today' (append)

Task 3/5: scada_today
✅ SQL 'scada_today' → 'scada_today' (append)

Task 4/5: duid
Table duid exists. Skipping (mode='ignore')
✅ SQL 'duid' → 'duid' (ignore)

Task 5/5: summary__incremental
✅ SQL 'summary__incremental' → 'summary' (append)

✅ All tasks completed successfull

True

In [1]:
import duckrun
con = duckrun.connect("temp/temp.lakehouse")
con.sql("""  FROM read_csv_auto('https://data.wa.aemo.com.au/datafiles/post-facilities/facilities.csv')
       """).write.mode("overwrite").saveAsTable("wa.base")
con.sql(" from base ").show(max_width=120)

Connecting to Lakehouse...
ℹ️  No schema specified. Using default schema 'dbo' for operations.
   Scanning all schemas for table discovery...

Please login to Azure CLI
🔍 Discovering schemas...
   Found 1 schemas: dbo

🔍 Discovering tables...

📊 Found 1 Delta tables. Attaching as views...

  ✓ Attached: dbo.sss → dbo_sss

✅ Successfully attached 1/1 tables


💡 Note: Tables are prefixed with schema (e.g., dbo_tablename)
   Default schema for operations: dbo

Writing to Delta table: wa.base (mode=overwrite)
✅ Table wa.base created/overwritten
┌──────────────────┬──────────────────────┬───┬──────────────────┬──────────────────────┬─────────────────────┐
│ Participant Code │   Participant Name   │ … │ System Size (MW) │ Remaining capacity…  │    Extracted At     │
│     varchar      │       varchar        │   │      double      │        double        │      timestamp      │
├──────────────────┼──────────────────────┼───┼──────────────────┼──────────────────────┼─────────────────────┤
│ ALC

In [2]:
con.copy("C:\lakehouse\default\Files\calendar","xxx")

📁 Uploading 2 files from 'C:\lakehouse\default\Files\calendar' to OneLake Files...
   Target folder: xxx
  ✓ Uploaded: C:\lakehouse\default\Files\calendar\calendar.parquet → xxx/calendar.parquet (13.2 KB)
  ✓ Uploaded: C:\lakehouse\default\Files\calendar\data_0.parquet → xxx/data_0.parquet (0.1 KB)

✅ Successfully uploaded all 2 files to OneLake Files


True

In [4]:
con.download("xxx","C:\lakehouse\default\Files\calendar",overwrite=True)

📁 Discovering files in OneLake Files...
   Source folder: xxx
📥 Downloading 2 files to 'C:\lakehouse\default\Files\calendar'...
  ✓ Downloaded: xxx/calendar.parquet → C:\lakehouse\default\Files\calendar\calendar.parquet (13.2 KB)
  ✓ Downloaded: xxx/data_0.parquet → C:\lakehouse\default\Files\calendar\data_0.parquet (0.1 KB)

✅ Successfully downloaded all 2 files from OneLake Files


True